# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [17]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots

In [18]:
include("../../../src/decompositions/naive_SVD.jl")
include("../../../src/decompositions/curvature_reweighed_SVD.jl")

curvature_reweighed_SVD (generic function with 1 method)

### Load data and construct manifold ###

In [19]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100


100

In [20]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[1]

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


100-element Vector{Matrix{Float64}}:
 [0.3986987613875975 -0.004674981341706422 0.026595219096028272; -0.004674981341706429 0.9714614552799347 0.019376401778023734; 0.026595219096028216 0.019376401778023765 0.9148073014173103]
 [0.23592164327709567 0.007033351603041819 0.0014652185807598611; 0.007033351603041833 0.9196298245118651 -0.011535996092956635; 0.0014652185807598611 -0.011535996092956607 0.9893676079102491]
 [6.505451123922677 -0.08270255365377853 0.24269335144903348; -0.08270255365377852 0.994482631724813 0.029323341051963697; 0.2426933514490336 0.029323341051963752 0.9650121044518502]
 [3.2663543472007364 0.02034254595516269 -0.10322665455557445; 0.02034254595516269 1.0280576882545684 -0.011539359776212847; -0.10322665455557445 -0.011539359776212791 1.0196541929444223]
 [1.4491174736185202 0.07540603148483727 0.06946533407078842; 0.07540603148483732 0.9650222418878447 0.020452583075566855; 0.06946533407078836 0.020452583075566855 1.0013347147077842]
 [1.2990868047004291 -0.0

In [21]:
# Export slice image
asymptote_export_SPD("results/artificial1D_orig.asy", data=data);

### Construct low rank approximation ###

In [22]:
q = mean(M, data)
# q = 1. * Matrix(I, 3, 3)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [23]:
N_R_q, N_U = naive_SVD(M,q, data);
CR_R_q, CR_U = curvature_reweighed_SVD(M,q, data);

In [24]:
ref_distance = sqrt(sum(distance.(Ref(M), Ref(q), data).^2))

N_tangent_distances_r = zeros(d)
N_distances_r = zeros(d)
CR_tangent_distances_r = zeros(d)
CR_distances_r = zeros(d)

for rank in 1:d
    N_log_q_data_r = Symmetric.([sum([N_R_q[i] * N_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    CR_log_q_data_r = Symmetric.([sum([CR_R_q[i] * CR_U[k,i] for i in d-rank+1:d]) for k in 1:n])
    # expoentiate back
    N_data_r = exp.(Ref(M), Ref(q), N_log_q_data_r)
    CR_data_r = exp.(Ref(M), Ref(q), CR_log_q_data_r)


    # compute relative tangent space error
    N_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - N_log_q_data_r).^2)) / ref_distance
    CR_tangent_distances_r[rank] = sqrt(sum(norm.(Ref(M), Ref(q),  log_q_data - CR_log_q_data_r).^2)) / ref_distance


    # compute relative manifold error
    N_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, N_data_r).^2)) / ref_distance
    CR_distances_r[rank] = sqrt(sum(distance.(Ref(M), data, CR_data_r).^2)) / ref_distance
end

In [25]:
plot(1:d, [N_tangent_distances_r, CR_tangent_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/artificial1D_tangent_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_tangent_rank.png"

In [26]:
plot(1:d, [N_distances_r, CR_distances_r], label = ["standard inner product" "reweighted inner product"], ylims=(0,1), xlims=(1,d))
savefig("results/artificial1D_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_rank.png"